<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [0]:
# Import any packages you need here
# numpy for processing arrays
import numpy as np
# copy used when I want to replicate a list without affecting the original list
import copy
# Use Queue for A* search
from queue import PriorityQueue
# to print table
import pandas as pd
# measure time running
import time
# Use itertools to iterate through the permutations
import itertools

In [0]:
#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    # YOUR CODE HERE

    def __init__(self, current_state, gval, hval, parent = None, explored = False):
      # input: current_state, gval (current cost), hval (heuristic cost)
      #   (optional input): parent: the parent state (for backtracking)
      #   (optional input): explored: check if the node has been explored or not
      self.current_state = current_state
      self.cost = gval + hval  # cost total
      self.curr_cost = gval #current cost
      self.heur_cost = hval #heuristic cost
      self.parent = parent
      self.explored = explored

    def __lt__(self, other):
      # Comparison function based on cost
      return self.cost < other.cost

    def __str__(self):
      return str(self.current_state)

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [0]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # YOUR CODE HERE
    # flatten the array and put it again as a list
    flat_state = list(np.array(state).flatten())

    # counting the number of misplaced tiles
    count = 0
    # run through the list
    for i in flat_state:
      # we know that when we flatten the list out, for the goal state, the value 
      # at a certain tile has to match the index of that tile 
      if i != flat_state.index(i) and i!=0:
        count += 1
    return count


# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # the mahattan distance
    count = 0
    # run through each element of the state
    for row in range(len(state)):
      for col in range(len(state)):
        value = state[row][col]
        # every tile except the empty tile 
        if value != 0:
          # row_pos and col_pos are the correct position of the tile
          row_pos = value//len(state)
          col_pos = value%len(state)
          # absolute distance
          count += abs(row_pos - row) + abs(col_pos - col)  
    return count  
    
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
  # use the pattern database db1 and db2
  global db1, db2
  if len(state)!=3:
    raise ValueError("only work for 3*3 dims")
  # make the current state into list
  flat_state = np.array(state).flatten().tolist()
  # find the index of the elements required to plug in the pattern design
  index1 = []
  index2 = []
  item1 = [0,1,2,3,4]
  item2 = [0,5,6,7,8]
  # each db requires 5 elements -> we are finding the key here
  for i in range(5):
    index1.append(flat_state.index(item1[i]))
    index2.append(flat_state.index(item2[i]))
  index1 = tuple(index1)
  index2 = tuple(index2)
  # find the maximum cost
  return max(db1[index1], db2[index2])

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [0]:
# Import any packages or define any helper functions you need here

# YOUR CODE HERE (OPTIONAL)
# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
  """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
  Inputs:
      -state: The initial state of the puzzle as a list of lists
      -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
  Outputs:
      -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
      -exp: The number of nodes expanded to reach the solution
      -max_frontier: The maximum size of the frontier over the whole search
      -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                  that represents the state of the board at the ith step of the solution.
      -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
        if the state is not solvable, then return -2
  """
  # YOUR CODE HERE
  # check validity
  if not validity_check(state):
    return 0, 0, 0, [], -1
   
  if not solvable_check(state):
    return 0, 0, 0, [], -2


  #Initialize node
  start_node = PuzzleNode(state, 0, heuristic(state))

  # Frontier, stored as a Priority Queue to maintain ordering
  frontier = PriorityQueue()
  frontier.put(start_node)

  goal_node = goal_state(len(state))
  exp = 0  #expansion
  max_frontier = 0 #maximum of the frontier
  opt_path = []  # to store the optimal path

  # store info of the travelled nodes
  history_cost = {str(start_node): start_node}
    
  while not frontier.empty():
    # Take the next available node from the priority queue
    cur_node = frontier.get()

    # move on if explored
    if cur_node.explored:
      continue

  # goal state
    if cur_node.current_state == goal_node:
      break

    # generate possible next steps
    possible_states = next_step(cur_node.current_state)

    for state in possible_states:
      # 1 cost for expansion
      cur_cost = cur_node.curr_cost + 1

      if str(state) in history_cost:
        if history_cost[str(state)].curr_cost > cur_cost:
          history_cost[str(state)].explored = True
        else:
          continue
        
      # find the heuristic cost
      hval = heuristic(state)
      # add new node to the frontier
      next_node = PuzzleNode(state, cur_cost, hval, cur_node)
      frontier.put(next_node)
      history_cost[str(next_node)] = next_node
    exp += 1

    # get max frontier
    max_frontier = max(max_frontier, frontier.qsize())
    
  #######
  # finding's over: now we travel backward to find the optimal path
  # start from the current node (end node)
  opt_path.append(cur_node.current_state)
  # while there are parents, go into that direction
  while cur_node.parent:
    opt_path.append(cur_node.parent.current_state)
    cur_node = cur_node.parent
  
  # as we are appending to the list, now we need to reverse it
  opt_path.reverse()
  steps = len(opt_path) - 1 # as we does not count the initial state
  return steps, exp, max_frontier, opt_path, 0 
    

# check if the input is proper or not
def validity_check(state):

  # Input: the state
  # Output: the validity statement (True/False) of the state: False if not valid

  # get the number of rows
  num_row = len(state)
  array_state = np.array(state)

  # there are 2 tests:
  # 1st test is: whether the shape fits num_row*num_row (Square shape)
  # 2nd test is: whether elements in the list are really from 0 to num_row**2

  # initialize the variables
  size_check = False
  value_check = False

  # check the size
  if array_state.shape == (num_row, num_row):
    size_check = True

  # check the elements (by flattening the state and sort them)
  if sorted(list(array_state.flatten())) == list(range(num_row**2)):
    value_check = True

  # to be valid, both size_check and value_check must be True
  return size_check and value_check

def solvable_check(state):
  # Input: state
  # Output: solvability

  flat_state = np.array(state).flatten()
  count = 0

  # as described in the extension part, we are counting the inversions
  for i in range(len(flat_state)):
    if flat_state[i] != 0:
      for k in range(i+1, len(flat_state)):
        # count the number of non-empty tiles have smaller value but bigger index 
        if (flat_state[k] != 0) and (flat_state[k] < flat_state[i]):
          count += 1

  # get the row index of ethe empty tie
  idx_0 = list(flat_state).index(0)
  row_num_0 = idx_0//3

  # n = even: inversion + row_index remains even for solvability
  if len(state) % 2 == 0:
    if (count + row_num_0) % 2 == 0: #  correct state
      return True
    else: # unsolvable state
      return False

  # n = odd: inversion remains even for solvability
  else:
    if (count) % 2 == 0: #  correct state
      return True
    else: # unsolvable state
      return False

def goal_state(n):
  # Input: the size of the state (number of row)
  # Output: the goal state for n*n matrix
  # create a goal state to compare with the outcome
  # the outcome will be a list of lists

  goal = []
  # for each row, we fill in the number from i*n to (i+1)*n-1
  # for example: row 2 in 4*4 will be 8,9,10,11: from 8=2*4 to 11=(3*4-1)
  for i in range(n):
    goal.append(list(range(i*n, (i+1)*n)))
  return goal

def next_step(state):
  # Input: the state
  # Output: the list of possible next steps

  # First, find the empty tile:
  array_state = np.array(state)
  pos_0 = np.where(array_state == 0)
  row_0, col_0 = pos_0[0][0], pos_0[1][0]
  
  possible_steps = []

  # create an item by moving the empty tile up (move the tile above the empty tile down)
  if row_0 != 0: 
    # deep copy in case it messes up the original state
    copied_items = copy.deepcopy(state)
    # create an item above   
    copied_items[row_0][col_0], copied_items[row_0-1][col_0] = copied_items[row_0-1][col_0], copied_items[row_0][col_0]
    possible_steps.append(copied_items)
  
  # create an item by moving the empty tile down (move the tile above the empty tile up)
  if row_0 != len(state) - 1:
    # deep copy in case it messes up the original state
    copied_items = copy.deepcopy(state)
    # create an item above   
    copied_items[row_0][col_0], copied_items[row_0+1][col_0] = copied_items[row_0+1][col_0], copied_items[row_0][col_0]
    possible_steps.append(copied_items)

  # create an item by moving the empty tile left (move the tile on the left of the empty tile right)
  if col_0 != 0:
    # deep copy in case it messes up the original state
    copied_items = copy.deepcopy(state)
    # create an item above   
    copied_items[row_0][col_0], copied_items[row_0][col_0-1] = copied_items[row_0][col_0-1], copied_items[row_0][col_0]
    possible_steps.append(copied_items)

  # create an item by moving the empty tile right (move the tile on the right of the empty tile left)
  if col_0 != len(state) - 1:
    # deep copy in case it messes up the original state
    copied_items = copy.deepcopy(state)
    # create an item above   
    copied_items[row_0][col_0], copied_items[row_0][col_0+1] = copied_items[row_0][col_0+1], copied_items[row_0][col_0]
    possible_steps.append(copied_items)

  return possible_steps

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [0]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [0]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [0]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [0]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

#### **1. Initial state solvability:**

We can prove that there's a characteristic that distinguish solvable and unsolvable cases. 

When we flatten the state, call Y(x) the number of values after x in the list and have smaller values. Call the sum of all Y(x) for every x in the list to be the inversion value of the state. One property is that for n*n table:
- If n is odd: inversion value%2 remains constant through valid moves
- If n is even, the sum of  inversion value%2 and the row_index of the empty tiles remains constant through valid moves

**We will prove for n odd first:**
- A horizontal slide won't change inversions
- A vertical slide (without losing generality, a downward slide) of value $a_0$ will affect n-1 values: $a_1, a_2, ..., a_{n-1}$ between the initial and new positions (from $a_0,a_1,...,a_{n-1} \rightarrow a_1, ..., a_{n-1}, a_0$). Call the number of values $> a_0$ to be $m$. Then when we move $a_0$ to the back, the inversion values will increase by $m$ (as these $m$ values will have an additional value smaller but stay after them in the list). However, there will still be $n-1-m$ values smaller than $a_0$. Then inversions value will decrease by $n-1-m$ (as $a_0$ initially has these $n-1-m$ values smaller but stay after  in the list, but not anymore). $$\Rightarrow inversion\_values = inversion\_values + m - (n-1-m) = inversion_values + 2m - (n-1)$$As $n$ odd, $n-1$ is even => inversion values change by an even amount, which its odd/even property holds. 
- The goal state has the inversion value = 0 (as everything is perfectly ordered). Hence, if the initial state has an odd inversion value, it will be unsolvable.


**We will prove for n even:**
- A horizontal slide won't change inversions or the row index of the empty tile. 
- A vertical slide (without losing generality, a downward slide) of value $a_0$ will affect n-1 values: $a_1, a_2, ..., a_{n-1}$ between the initial and new positions (from $a_0,a_1,...,a_{n-1} \rightarrow a_1, ..., a_{n-1}, a_0$). Call the number of values $> a_0$ to be $m$. Then when we move $a_0$ to the back, the inversion values will increase by $m$ (as these $m$ values will have an additional value smaller but stay after them in the list). However, there will still be $n-1-m$ values smaller than $a_0$. Then inversions value will decrease by $n-1-m$ (as $a_0$ initially has these $n-1-m$ values smaller but stay after  in the list, but not anymore). $$\Rightarrow inversion\_values = inversion\_values + m - (n-1-m) = inversion_values + 2m - (n-1)$$As $n$ even, $n-1$ is odd => inversion values change by an odd amount. At the same time, sliding vertically will change the row_index by 1 or -1, which is an odd amount. Hence, the sum of inversion values and row_index will change by an even amount, making the sum's odd/even property holds. 
- The goal state has the inversion value = 0 (as everything is perfectly ordered) and the row index of empty tile = 0 $\Rightarrow$ sum = 0. Hence, if the initial state has the sum of inversion values and empty row index to be odd, it will be unsolvable.

This proof was implemented in the code already as $solvable\_check()$


### **2. Extraheuristics & Memoization**

Create a pattern database (memoization) and create a heuristic based on that. 

We can create an admissible heuristic for this problem—one that does not overestimate: solving the relaxed version of this problem. We will create a pattern database for 2 cases: case one is with 0, 1, 2, 3, 4 at the front, and case two is with 0 at the front and 5, 6, 7, 8 at the end. For each configuration, we measure the number of steps to reach these relaxed state and choose the max of those 2 values. These 2 are relaxed problems for the real goal, and they don't overestimate the number of steps. We will also calculate the costs at possible states and store them in a dictionary. This method (memoization) will enhance the time for running this new heuristic. 

The reason we choose these 2 configurations is that
- 0 must be in the setup (as it's the empty tile)
- I divide the 8 tiles into two groups, and 1-2-3-4 and 5-6-7-8 are the two simple groups. If I left out any value (for example, only use the group 1-2-3-4), the heuristic could make the state converge to states with 1-2-3-4  at the beginning, but then it has to explore every possible configurations later on (which will lead to more nodes in the frontier). 


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [0]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

##### **Code for extension 2 and 3**

In [0]:
# This code is to solve the sub_problem: the relaxed version
def solve_sub_Puzzle(state, heuristic, goal_node):
  """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
  Inputs:
      -state: The initial state of the puzzle as a list of lists
      -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
      - goal_node: the goal state (as we have 2 relaxed problems, customized goal_node will do the trick)
  Outputs:
      -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
  """

  #Initialize node
  start_node = PuzzleNode(state, 0, heuristic(state))

  # Frontier, stored as a Priority Queue to maintain ordering
  frontier = PriorityQueue()
  frontier.put(start_node)

  exp = 0  #expansion
  max_frontier = 0 #maximum of the frontier
  opt_path = []  # to store the optimal path


  # store info of the travelled nodes
  history_cost = {str(start_node): start_node}
    
  while not frontier.empty():
    # Take the next available node from the priority queue
    cur_node = frontier.get()

    # move on if explored
    if cur_node.explored:
      continue

  # goal state
    if cur_node.current_state == goal_node:
      break

    # generate possible next steps
    possible_states = next_step(cur_node.current_state)

    for state in possible_states:
      # 1 cost for expansion
      cur_cost = cur_node.curr_cost + 1

      if str(state) in history_cost:
        if history_cost[str(state)].curr_cost > cur_cost:
          history_cost[str(state)].explored = True
        else:
          continue
        
      # find the heuristic cost
      hval = heuristic(state)
      # add new node to the frontier
      next_node = PuzzleNode(state, cur_cost, hval, cur_node)
      frontier.put(next_node)
      history_cost[str(next_node)] = next_node
    exp += 1

    # get max frontier
    max_frontier = max(max_frontier, frontier.qsize())
    
  #######
  # finding's over: now we travel backward to find the optimal path
  # start from the current node (end node)
  opt_path.append(cur_node.current_state)
  # while there are parents, go into that direction
  while cur_node.parent:
    opt_path.append(cur_node.parent.current_state)
    cur_node = cur_node.parent
  
  # as we are appending to the list, now we need to reverse it
  opt_path.reverse()
  steps = len(opt_path) - 1 # as we does not count the initial state
  return steps

In [0]:
def create_db_1():
  # Case 1: having 0, 1, 2, 3, 4
  # output: the dictionary of costs of relaxed subproblems

  # get the index for 0, 1, 2, 3, 4 (all possile index)
  subproblems = itertools.permutations(range(9), 5)
  db1 = {}

  # run though all permutation
  for subproblem in subproblems:
    # a list with random big number - bigger than 8
    initial_state = [100]*9

    # using the index to plug values into list
    for i in range(5):
      initial_state[subproblem[i]] = i
    
    # reshape
    state = np.reshape(np.array(initial_state), (3,3)).tolist()
    # solve the relaxed version + we provide the desired goal state
    cost = solve_sub_Puzzle(state, heuristics[1], [[0,1,2],[3,4,100],[100,100,100]])
    # add to dictionary
    db1[subproblem] = cost
  return db1

def create_db_2():
  # Case 1: having 0, 5, 6, 7, 8
  # output: the dictionary of costs of relaxed subproblems

  # get the index for 0, 5, 6, 7, 8 (all possile index)
  subproblems = itertools.permutations(range(9), 5)
  db2 = {}
  substitute = [0,5,6,7,8]

  # run through all permutations
  for subproblem in subproblems:
    # a random value than > 0 but smaller than 5 
    initial_state = [0.5]*9

    # plug in values
    for i in range(5):
      initial_state[subproblem[i]] = substitute[i]
    # reshape the list
    state = np.reshape(np.array(initial_state), (3,3)).tolist()
    # solve the relaxed version + we provide the desired goal state
    cost = solve_sub_Puzzle(state, heuristics[1],  [[0,0.5,0.5],[0.5,0.5,5],[6,7,8]])
    # add to dictionary
    db2[subproblem] = cost
  return db2

db1 = create_db_1()
db2 = create_db_2()

In [0]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [0]:
## Memoization test - will be carried out after submission

#### **Test and compare efficiency**

In [14]:
# for 3 test cases
for i in range(3):
  a_1 = time.time()
  # run through h3
  steps_new, expansions_new, max_frontier_new, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
  a_2 = time.time()
  # run through h2
  steps_man, expansions_man, max_frontier_man, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
  a_3 = time.time()
  # run through h1
  steps_mts, expansions_mts, max_frontier_mts, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
  a_4 = time.time()
  # calculate the time
  time_new, time_man, time_mts = a_2-a_1, a_3-a_2, a_4-a_3

  print("Comparison for case", i+1)
  table = pd.DataFrame([["steps", steps_mts, steps_man, steps_new],
                        ["expansion", expansions_mts, expansions_man, expansions_new],
                        ["max_frontier", max_frontier_mts, max_frontier_man, max_frontier_new],
                        ["time", time_mts, time_man, time_new]],
                        columns = ["criteria", "h1", "h2", "h3"])
  print(table)
  print("-------------------------------------")

Comparison for case 1
       criteria          h1         h2         h3
0         steps   17.000000  17.000000  17.000000
1     expansion  883.000000  84.000000  45.000000
2  max_frontier  557.000000  51.000000  40.000000
3          time    0.108166   0.007059   0.006295
-------------------------------------
Comparison for case 2
       criteria            h1           h2         h3
0         steps     25.000000    25.000000  25.000000
1     expansion  25349.000000  1703.000000  97.000000
2  max_frontier  12075.000000   957.000000  89.000000
3          time      3.011866     0.151177   0.015969
-------------------------------------
Comparison for case 3
       criteria            h1           h2          h3
0         steps     28.000000    28.000000   28.000000
1     expansion  63972.000000  1459.000000  252.000000
2  max_frontier  21725.000000   887.000000  202.000000
3          time      7.144778     0.162104    0.026756
-------------------------------------


#### **General comment on the new heuristics**

We can see, the new heuristics performs much better in expansions, max_frontier and time. This means that the new heuristic expand less nodes (smaller expansions) than the others heuristic, indicating better efficient heuristics. In conclusion, h3 > h2 > h1. 

We only implement for 3x3 because of the memory. For 3x3, each pattern database already stored $9P5 = 15120$ permutations (select 5 tiles in 9 tiles, with order). In total for 2 databases are 30240 data points. Storing and calculating the $solve\_sub\_Puzzle$ for each step is already a lot for memory and time. If we extend for 4x4 and we do 7-8 separations plus the empty tile: in total, we'll have: $16P8 + 16P9 = 518918400 + 4151347200 = 4670265600 $. 4.6 Billion data cases to run and to store is too much to handle. Hence, this approach for pattern database is only efficient for 3*3 or smaller.